<a href="https://colab.research.google.com/github/con11235/covid19_visualize/blob/master/markdown/%EC%8A%B9%ED%98%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part.0. 분석의 목적 및 흐름
안녕하세요, 팀 클릭입니다. 코로나 발생 이후 사회에 생긴 변화는 새로운 소비 트렌드가 등장했다는 점입니다. 우리는 이와 관련한 데이터를 분석하고 파악함으로써 이러한 트렌드를 가장 적극적으로 반영할 수 있는 산업이 무엇인지를 찾고자 합니다.  
 
코로나로 인해 생긴 가장 큰 변화를 떠올려보면 다음과 같은 두 특징이 있습니다.  
 
첫 번째, 코로나로 인해 발생한 여러 소비 트렌드들로 인해 사람들이 주로 소비하는 산업 자체가 달라졌다는 점입니다.  
![코로나19와 5대 소비 트렌드 이미지](https://i.ibb.co/VVfwcS0/1.png)  
집에서 보내는 시간의 증가로 집 안에서의 다양한 의식주 라이프스타일과 여가생활을 추구하는 '홈코노미', 인간 본원적 가치를 중시하는 현상으로 인해 불안감 및 두려움 등의 심리를 위안하는 '불안케어', 나를 위한 자기중심적 소비인 '에고이즘' 등이 대표적인 예입니다[(참고 기사)](http://m.joseilbo.com/news/view.htm?newsid=402010#_enliple). 우리는 소비 트렌드의 변화로 영향을 받았을 산업을 찾고자 하였고, 이를 위해 카드 소비 데이터를 분석하기로 하였습니다.  
 
두 번째, 대면 접촉이 꺼려짐으로 인해 비대면 방식의 생활이 확산되었습니다. 우리가 이러한 비대면 생활에 쉽게 적응될 수 있었던 것은 스마트폰이나 인터넷의 힘이 큽니다. 이로 미루어 보아, 사람들이 코로나 이후로 관심이 높아진 산업이 무엇인지를 찾기 위한 답은 스마트폰 안에 있을 것입니다. 우리는 이를 찾기 위해 스마트폰 앱의 순위 데이터를 장르별로 분석하기로 하였습니다.

즉, 우리는 코로나로 인한 소비 및 흥미의 변화를 파악하여 트렌드가 반영된 유망 산업을 찾는 방식으로 데이터를 분석해보았습니다.  
 
이후의 설명에는 필수적인 코드와 결과만을 포함하며 전체 코드에 대한 정보는 [깃허브 링크](https://github.com/con11235/covid19_visualize)에서 확인할 수 있습니다.

# Part.1. 코로나 확진자 데이터의 분석  
주어진 데이터 내에 한계가 있다보니 우리는 분석 대상을 서울시로 한정짓기로 하였습니다.  
위에서 언급한 데이터들을 분석해보기 이전에, 코로나 자체의 발생 동향을 파악하기 위해 서울시 확진자의 데이터를 분석해보기로 하였습니다.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' 
!apt -qq -y install fonts-nanum
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

import datetime
import plotly.express as px

import plotly
from plotly.graph_objs import Scatter, Line, Layout
 
import importlib
import json
 
from plotly.subplots import make_subplots
import plotly.graph_objects as go

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [4]:
patientinfo=pd.read_csv('/content/gdrive/My Drive/dacon/COVID_19/PatientInfo.csv')
patientinfo.head(3)

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,male,50s,Korea,서울,강서구,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,male,30s,Korea,서울,중랑구,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,male,50s,Korea,서울,종로구,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaN,released


### 서울 확진자만 분류


In [5]:
patientinfo_seoul=patientinfo[patientinfo['province']=='서울']

In [6]:
patient_seoul=patientinfo_seoul[['province','city','confirmed_date']]
patient_seoul['total_ads']=patient_seoul['province']+' '+patientinfo_seoul['city']

서울 내의 지역별 분류

In [9]:
patient_seoul=patient_seoul[['confirmed_date','total_ads']]
patient_seoul['cnt']=1
patient_seoul=patient_seoul.sort_values(by=['confirmed_date',],ascending=True)
patient_seoul.head(3)

,confirmed_date,total_ads,cnt
0,2020-01-23,서울 강서구,1
1,2020-01-30,서울 중랑구,1
2,2020-01-30,서울 종로구,1


날짜별 지역 확진자 수로 병합 (ex, 1/23일 종로구가 2번 -> 2명)

In [10]:
patient_seoul=patient_seoul.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

지역별 누적 확진자 추가

In [12]:
patient_seoul['cnt_sum']=patient_seoul.groupby('total_ads')['cnt'].cumsum()

In [13]:
fig=px.line(patient_seoul,x='confirmed_date',y='cnt_sum',color='total_ads')
fig.update_layout(
    title={
        'text': "지역별 확진자 변화(서울 한정)",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

### 1월/2월초/2월말/3월초/3월말/4월초/4월말/5월초/5월말/6월초 나누기

In [14]:
is_date=patient_seoul['confirmed_date']<'2020-02-01'
patient_seoul_jan=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)

patient_seoul_jan['confirmed_date']='1월 말'
patient_seoul_jan=patient_seoul_jan.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

In [15]:
#2월 초
is_date_1=patient_seoul_1['confirmed_date']<'2020-02-15'
patient_seoul_feb_1=patient_seoul[is_date_1]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_feb_1['confirmed_date']='2월 초'
patient_seoul_feb_1=patient_seoul_feb_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#2월 말
is_date=patient_seoul['confirmed_date']<'2020-03-01'
patient_seoul_feb_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_feb_2['confirmed_date']='2월 말'
patient_seoul_feb_2=patient_seoul_feb_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#3월 초
is_date=patient_seoul['confirmed_date']<'2020-03-16'
patient_seoul_mar_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_mar_1['confirmed_date']='3월 초'
patient_seoul_mar_1=patient_seoul_mar_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#3월 말
is_date=patient_seoul['confirmed_date']<'2020-04-01'
patient_seoul_mar_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_mar_2['confirmed_date']='3월 말'
patient_seoul_mar_2=patient_seoul_mar_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#4월 초

is_date=patient_seoul['confirmed_date']<'2020-04-16'
patient_seoul_apr_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_apr_1['confirmed_date']='4월 초'
patient_seoul_apr_1=patient_seoul_apr_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#4월 말
is_date=patient_seoul['confirmed_date']<'2020-05-01'
patient_seoul_apr_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_apr_2['confirmed_date']='4월 말'
patient_seoul_apr_2=patient_seoul_apr_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#5월 초
is_date=patient_seoul['confirmed_date']<'2020-05-16'
patient_seoul_may_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_may_1['confirmed_date']='5월 초'
patient_seoul_may_1=patient_seoul_may_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#5월 말
is_date=patient_seoul['confirmed_date']<'2020-06-01'
patient_seoul_may_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_may_2['confirmed_date']='3월 말'
patient_seoul_may_2=patient_seoul_may_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#6월 초

is_date=patient_seoul['confirmed_date']<'2020-06-16'
patient_seoul_june_1=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_june_1['confirmed_date']='6월 초'
patient_seoul_june_1=patient_seoul_june_1.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()

#6월 말
is_date=patient_seoul['confirmed_date']<'2020-07-01'
patient_seoul_june_2=patient_seoul[is_date]
patient_seoul=patient_seoul.drop(patient_seoul[is_date].index)
patient_seoul_june_2['confirmed_date']='6월 말'
patient_seoul_june_2=patient_seoul_june_2.groupby(['confirmed_date','total_ads'],as_index=False).cnt.sum()


NameError: ignored

In [ ]:
patient_seoul_ads=pd.concat([patient_seoul_jan,patient_seoul_feb_1,patient_seoul_feb_2,patient_seoul_mar_1,patient_seoul_mar_2,
                             patient_seoul_apr_1,patient_seoul_apr_2,patient_seoul_may_1,patient_seoul_may_2,
                             patient_seoul_june_1,patient_seoul_june_2],axis=0)

In [ ]:
patient_seoul_ads['cnt_sum']=patient_seoul_ads.groupby('total_ads')['cnt'].cumsum()

In [ ]:
fig=px.line(patient_seoul_ads,x='confirmed_date',y='cnt_sum',color='total_ads')
fig.update_layout(
    title={
        'text': "지역별 확진자 변화(서울 한정)",
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

### 집단감염 시각화

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
        
import seaborn as sns
import matplotlib.pyplot as plt

import folium 
from folium import plugins
# from folium.plugins import HeatMap

import json 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

import webbrowser

In [18]:
fpopl = pd.read_csv('/content/gdrive/My Drive/dacon/fpopl.csv')
case = pd.read_csv('/content/gdrive/My Drive/dacon/COVID_19/Case.csv')

FileNotFoundError: ignored

데이터로부터 발생 지역, 감염유형, 확진자 수, 위도, 경도를 확인할 수 있습니다.  
맨 마지막의 size값은 바로 아래에서 설정한 대/중/소규모 집단 감염의 분류에 따라 대규모는 1, 중규모는 2, 소규모는 3으로 라벨링 된 값입니다.

In [ ]:
data = case[['province','city','infection_case','confirmed','latitude','longitude','size']].dropna()
data

확진환자 수에 따라 집단감염을 분류하기 위해 확진환자 수의 분포를 막대그래프로 표현하였습니다.  
확진환자 수는 아래와 같이 80, 25인 지점에서 큰 격차를 보였습니다. 따라서 이들을 기준으로 하여 80명 이상은 **대규모 집단 감염**, 25명 이상 80명 미만은 **중규모 집단 감염**, 25명 미만은 **소규모 집단 감염**으로 분류합니다.

In [ ]:
data.sort_values(by='confirmed')
icdata = data[['infection_case','confirmed']][1:]
sns.barplot(x='infection_case',y='confirmed',data=icdata)
plt.xticks(rotation=60, fontsize = 5)
plt.axhline(y=80, color='gray', linewidth=1)
plt.axhline(y=25, color='gray', linewidth=1)
plt.show()

분류한 집단별로 발생 위치와 감염의 크기를 지도에 시각화 하기 위해 세개의 데이터로 분류합니다.  
생성한 데이터를 사용하여 지도에 시각화합니다.  
규모에 따라 색의 진하기와 원의 크기를 다르게 하여 비교할 수 있도록 하였습니다.  
분류별 감염 위치를 확인할 수 있도록 레이어를 사용하였습니다.

각 지역별 인구 수와 집단감염간의 관계를 살펴보기 위해 지역별 인구 분포를 지도에 시각화하였습니다. 이 부분은 [해당 링크](https://blog.naver.com/PostView.nhn?blogId=kcchang61&logNo=221350672356)를 참고하였습니다.

In [ ]:
code = []
fpopl1 = fpopl[['adstrd_code','popltn_cascnt','base_ymd','agrde_se_code']]\
.query('base_ymd in ("20200101")').dropna()
for i in range(len(fpopl1['adstrd_code'])):
    code.append(str(fpopl1['adstrd_code'][i]//1000))
print(len(code))
fpopl1['code'] = code

fpopl00 = fpopl1\
.query('agrde_se_code in ("age_00")').dropna()\
.groupby(['code'])['popltn_cascnt'].sum().reset_index()


In [ ]:
m = folium.Map(location=[37.55, 127], tiles="OpenStreetMap", zoom_start=10)
# Add the color for the chloropleth:

m.choropleth(
 geo_data=state_geo2,
 name='0~9세 유동인구수',
 data=fpopl00,
 columns=['code', 'popltn_cascnt'],
 key_on='feature.properties.SIG_CD',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.5,
 legend_name='Population Rate (%)'
)


fg1 = folium.FeatureGroup(name='대규모 집단감염')
m.add_child(fg1)

fg2 = folium.FeatureGroup(name='중규모 집단감염')
m.add_child(fg2)

fg3 = folium.FeatureGroup(name='소규모 집단감염')
m.add_child(fg3)

for i in range(len(data1)):
    tmp = data1.loc[i]
    folium.CircleMarker([tmp['latitude'],tmp['longitude']], radius=30, 
                        color='#a31c19',fill_color='a31c19',popup=tmp['infection_case']).add_to(fg1)

    
for i in range(len(data1),len(data1)+len(data2)):
    tmp = data2.loc[i]
    folium.CircleMarker([tmp['latitude'],tmp['longitude']], radius=20, 
                        color='#a63117',fill_color='#a63117',popup=tmp['infection_case']).add_to(fg2)

for i in range(len(data1)+len(data2),len(data1)+len(data2)+len(data3)):
    tmp = data3.loc[i]
    folium.CircleMarker([tmp['latitude'],tmp['longitude']], radius=10, 
                        color='#ed685e',fill_color='#ed685e',popup=tmp['infection_case']).add_to(fg3)

folium.LayerControl(collapsed=False).add_to(m)
    

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)

plugins.MousePosition().add_to(m) ## you can easily get coordinates.

m
